In [10]:
import pandas as pd
import numpy as np
import json


from urllib.parse import urlencode
from urllib.request import urlopen
import math
import time
import dateutil



endpoints = {"channel": 'channels', "video": "videos", "vid": "videos"}
#loading bot credentials
def get_credentials():
    try:
        file = open("input/config.json", "r")
    except FileNotFoundError:
        # no credentials found, create a holder file for user to add
        file = open("input/config.json", "a")
        file.write('{ \n "YOUTUBE_API_KEY\": \"INPUT_API_KEY_HERE\" \n}')
        file.close()

    file = open("input/config.json", "r")
    file = json.load(file)
    return file['YOUTUBE_API_KEY']
    

def send_endpoint(api_params, item_type, printable):
    api_endpoint = 'https://www.googleapis.com/youtube/v3/' + item_type
    encoded_params = urlencode(api_params)
    url = f'{api_endpoint}?{encoded_params}'
    if printable:
        print(url)
    
    with urlopen(url) as response:
        return [json.load(response)]


def get_videos_by_channel(api_token, channelId):
    #getting channel's playlist
    api_params = {
        'part' : "contentDetails",
        'key' : api_token,
        'id': channelId,
    }
    response = send_endpoint(api_params, "channels", False)
    uploads_playlist = response[0]['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    #getting uploads
    videoIds = []
    nextPageToken = None
    while(True):
        api_params = {
            'part' : "snippet",
            'key' : api_token,
            'playlist_id': uploads_playlist,
            'maxResult' : "25"
        }
        if(nextPageToken):
            api_params["pageToken"] = nextPageToken
        
        response = send_endpoint(api_params, "playlistItems", False)
        for item in response[0]["items"]:
            videoIds.append(item["snippet"]["resourceId"]["videoId"])
            
        try:
            nextPageToken = response[0]["nextPageToken"]
        except KeyError:
            nextPageToken = None
        if not nextPageToken:
            break
    return videoIds
    
    

def get_channel_by_video(api_token, video_id):
    api_params = {
        'part': "snippet",
        'key': api_token,
        'id' : video_id
    
    }
    response = send_endpoint(api_params, "videos", False)
    channelId = response[0]["items"][0]["snippet"]["channelId"]
    #statistics = response[0]["items"][0]["statistics"]
    return response[0]["items"][0]["snippet"]["channelId"]
def get_video_details(api_token, videoIds):
    video_deets = []
    for i in range(math.ceil(len(videoIds)/50)):

        video_query = ','.join(videoIds[50*i:(50*i)+50])
        api_params = {
            "key": YOUTUBE_API_KEY,
            "id": video_query,
            "part": "statistics, snippet"
        }
        response = send_endpoint(api_params, "videos", False)
        for vid in response[0]["items"]:
            unixTime = int(dateutil.parser.parse(vid["snippet"]["publishedAt"][:10]).timestamp())
            video_deets.append([vid["id"], vid["snippet"]["title"],int(vid["statistics"]["viewCount"]),unixTime])
    return pd.DataFrame(video_deets, columns=["id", "title","views", "unix"])
def unix_timeframe(years=0,months=0,weeks=0,days=0,hours=0):
    return int(time.time()) - (31556926*years + 2629743*months + 604800*weeks + 86400*days + 3600*hours)




# making it a script

if __name__ == '__main__':
    #args = parseargs()
    
    #arguments
    video_id_from_channel = "ArD7oyCiXYc"
    view_threshold = 200000
    timeframe = unix_timeframe(years=1)
    

    YOUTUBE_API_KEY = get_credentials()
    channelId = get_channel_by_video(YOUTUBE_API_KEY, video_id_from_channel)
    videoIds = get_videos_by_channel(YOUTUBE_API_KEY, channelId)
    video_df = get_video_details(YOUTUBE_API_KEY, videoIds)
    
    
    
    # date threshold
    filtered_df = video_df[video_df['unix'] > timeframe]
    
    #views threshold
    filtered_df = filtered_df[filtered_df["views"] > view_threshold]
    
    #sort by most views first
    filtered_df = filtered_df.sort_values(by=["views"], ascending=False)
    #sort by oldest first
    #filtered_df.sort_values(by=["date"], ascending=True)
    
    #print out remaining videos
    i = 1
    
    print("Top 0 in descending view count order: \n")
    #print only the top 10
    for ind,row in filtered_df[:10].iterrows():
        print(f'{i}. {row["title"]} | https://www.youtube.com/watch?v={row["id"]}')
        i+=1
    

Top 0 in descending view count order: 

1. Pokemon Black and White - Did You Know Gaming? Ft. Lockstin (Nintendo DS) | https://www.youtube.com/watch?v=M2508SFpqE0
2. Super Mario 64 - Did You Know Gaming? Ft. Seth Everman | https://www.youtube.com/watch?v=VEg-EUhT9eA
3. Obscure Zelda Facts (BotW, TP, MM, OoT) - Did You Know Gaming? Ft. @Zeltik | https://www.youtube.com/watch?v=aS0CRT6_W1A
4. Every Cancelled GameCube Game - Did You Know Gaming? Ft. Remix | https://www.youtube.com/watch?v=WQIRyLuuZCw
5. Every Cancelled Nintendo DS Game - Did You Know Gaming? Ft. Remix | https://www.youtube.com/watch?v=lr9KveO0yQU
6. Super Mario Sunshine - Did You Know Gaming? Feat. Remix | https://www.youtube.com/watch?v=3tcbqNnZPAg
7. Mario Kart 64 - Did You Know Gaming? Feat. Dazz (N64) | https://www.youtube.com/watch?v=YVnDDL4-Id0
8. Nintendo's Rarest Consoles - Game History Secrets | https://www.youtube.com/watch?v=yy0hZ2JZDfw
9. Super Mario 128 Ft. Dunkey - DidYouKnowGaming | https://www.youtube.com/

In [9]:
filtered_df

,id,title,views,unix
29,M2508SFpqE0,Pokemon Black and White - Did You Know Gaming?...,622747,1604127600
40,VEg-EUhT9eA,Super Mario 64 - Did You Know Gaming? Ft. Seth...,615097,1598770800
63,aS0CRT6_W1A,"Obscure Zelda Facts (BotW, TP, MM, OoT) - Did ...",594559,1589180400
59,WQIRyLuuZCw,Every Cancelled GameCube Game - Did You Know G...,581587,1591426800
48,lr9KveO0yQU,Every Cancelled Nintendo DS Game - Did You Kno...,557608,1596006000
37,3tcbqNnZPAg,Super Mario Sunshine - Did You Know Gaming? Fe...,493967,1600585200
52,YVnDDL4-Id0,Mario Kart 64 - Did You Know Gaming? Feat. Daz...,477563,1594623600
36,yy0hZ2JZDfw,Nintendo's Rarest Consoles - Game History Secrets,472557,1601103600
34,IU1IML3xlp0,Super Mario 128 Ft. Dunkey - DidYouKnowGaming,470956,1602313200
13,SZcrPM-jDqY,WorkBoy: Lost Game Boy Add-on FOUND After 28 Y...,454373,1608969600
